In [35]:
import pandas as pd
import os
import datetime as dt
import numpy as np

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error

In [3]:
pd.set_option('display.max_row', None)
pd.set_option('display.max_column', None)

In [4]:
data_abs_path = os.path.abspath('../data')

In [5]:
data_abs_path

'c:\\Users\\Coni\\Desktop\\estudio_cv\\curso_mlops\\mlops_course_repo\\data'

In [6]:
jan_path = os.path.join(data_abs_path, "yellow_tripdata_2023-01.parquet")
feb_path = os.path.join(data_abs_path, "yellow_tripdata_2023-02.parquet")

In [48]:
jan_df = pd.read_parquet(jan_path)
feb_df = pd.read_parquet(feb_path)

In [49]:
print(jan_df.shape)
print(feb_df.shape)

(3066766, 19)
(2913955, 19)


In [50]:
jan_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [51]:
jan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [52]:
# Compute of duration variable
jan_df['duration'] = round((jan_df.tpep_dropoff_datetime - jan_df.tpep_pickup_datetime).dt.total_seconds()/60,2)

In [53]:
# Compute of duration variable
feb_df['duration'] = round((jan_df.tpep_dropoff_datetime - jan_df.tpep_pickup_datetime).dt.total_seconds()/60,2)

In [54]:
jan_df.describe().T

,count,mean,min,25%,50%,75%,max,std
VendorID,3066766.0,1.730215,1.0,1.0,2.0,2.0,2.0,0.443848
tpep_pickup_datetime,3066766,2023-01-17 00:22:26.288164,2008-12-31 23:01:42,2023-01-09 16:21:57.250000,2023-01-17 08:42:29.500000,2023-01-24 16:26:27,2023-02-01 00:56:53,NaN
tpep_dropoff_datetime,3066766,2023-01-17 00:38:06.427874,2009-01-01 14:29:11,2023-01-09 16:37:06,2023-01-17 08:58:30.500000,2023-01-24 16:42:49,2023-02-02 09:28:47,NaN
passenger_count,2995023.0,1.362532,0.0,1.0,1.0,1.0,9.0,0.89612
trip_distance,3066766.0,3.847342,0.0,1.06,1.8,3.33,258928.15,249.583756
RatecodeID,2995023.0,1.49744,1.0,1.0,1.0,1.0,99.0,6.474767
PULocationID,3066766.0,166.398046,1.0,132.0,162.0,234.0,265.0,64.244131
DOLocationID,3066766.0,164.392631,1.0,114.0,162.0,234.0,265.0,69.943682
payment_type,3066766.0,1.194483,0.0,1.0,1.0,1.0,4.0,0.529476
fare_amount,3066766.0,18.367069,-900.0,8.6,12.8,20.5,1160.1,17.807822


In [55]:
feb_df.rename(columns={'Airport_fee':'airport_fee'}, inplace=True)

In [56]:
# Remove outiers
rm_outliers = lambda df: (df.duration>=1)&(df.duration<=60)
jan_df_rm_outliers = jan_df[rm_outliers]

In [57]:
len(jan_df_rm_outliers.duration)/len(jan_df.duration)

0.9812202822125979

In [58]:
# Remove outliers
feb_df_rm_outliers = feb_df[rm_outliers]

In [59]:
categorical_cols = ['PULocationID', 'DOLocationID']
target_col =  ['duration']

# Convert selected categorical columns into string, otherwise it will label encode them
jan_df_rm_outliers[categorical_cols] = jan_df_rm_outliers[categorical_cols].astype('str')

C:\Users\Coni\AppData\Local\Temp\ipykernel_16052\2469520457.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_df_rm_outliers[categorical_cols] = jan_df_rm_outliers[categorical_cols].astype('str')


In [63]:
# Turn the dataframe into a list of dictionaries
train_dict = jan_df_rm_outliers[categorical_cols].to_dict(orient='records')

# Creates an instance of DictVectorizer class and assigns it to dv
dv = DictVectorizer(sparse=True)

# Fit a dictionary vectorizer 
dv.fit(train_dict)

DictVectorizer()

In [64]:
# Feature matrix --> One hot encode the data with the DictVectorizer object
X_train = dv.transform(train_dict)

In [65]:
X_train.shape

(3009173, 515)

In [66]:
# Trainig a linear regression model
y_train = jan_df_rm_outliers[target_col]
lr = LinearRegression()
lr.fit(X_train, y_train)

# Prediction on train dataset
y_pred = lr.predict(X_train)

In [67]:
# RMSE on training data
rmse_train = np.sqrt(mean_squared_error(y_pred, y_train))

In [68]:
rmse_train

7.649261720998863

In [69]:
# Evaluating the model

# Test dataset
# Convert selected categorical columns into string
feb_df_rm_outliers[categorical_cols] = feb_df_rm_outliers[categorical_cols].astype('str')
test_dict = feb_df_rm_outliers[categorical_cols].to_dict(orient='records')
# We use the same object to transform validation
X_test = dv.transform(test_dict)

y_test = feb_df_rm_outliers[target_col]
y_pred = lr.predict(X_test)

rmse_test = np.sqrt(mean_squared_error(y_pred, y_test))

C:\Users\Coni\AppData\Local\Temp\ipykernel_16052\399907897.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb_df_rm_outliers[categorical_cols] = feb_df_rm_outliers[categorical_cols].astype('str')


In [70]:
rmse_test

11.873372011653485

In [44]:
import gc

In [71]:
del jan_df
del jan_df_rm_outliers
del feb_df
del feb_df_rm_outliers

In [72]:
gc.collect()

8